# Template

In [3]:
local = True

In [4]:
if not local:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

In [5]:
if not local:
    %cd /content/drive/MyDrive/Sun/ML Shock/Final project/

In [6]:
import sys
if local:
    sys.path.insert(0,"C:/Users/Amy/Desktop/Green_Git/eegClassification/utils")
else:
    sys.path.insert(0,"/content/drive/MyDrive/Sun/ML Shock/Final project/")

In [7]:
# Import libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
import os

from CustomDataLoaderNPY import CustomDataset
from torch.utils.data import DataLoader
from model_architectures import CustomCNN, TransNet_Resnet18, TransNet_Efficientnetb0


In [8]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    print("GPU count", torch.cuda.device_count())

CUDA is not available.  Training on CPU ...


In [9]:
# Type of input data
data_type = "spec" # "eeg_raw" #"eeg_spec"  #
# number of subprocesses to use for data loading
import multiprocessing as cpu
num_workers = 0 #cpu.cpu_count() #- 1
# how many samples per batch to load
batch_size = 64


In [16]:
# Load (train or test) data from csv file
if local:
    path = f"C:/Users/Amy/Desktop/Green_Git/eegClassification/notebooks/Data processing/data_prep_all_{data_type}/"
    path_df = "C:/Users/Amy/Desktop/Green_Git/eegClassification/files/"
else:
    path = f"./data_prep_all_{data_type}/"
    path_df = "./data/"


In [17]:
if data_type == "spec":
    input_shape = (3,400,299)
elif data_type == "eeg_spec":
    input_shape = (20,129,43)
elif data_type == "eeg_raw":
    input_shape = (20,9800)

In [18]:
data_path = path+'train/'
data_files = os.listdir(data_path)
data_files = [f.split('.')[0].split('_')[1] for f in data_files if f.startswith('images')]
N_items = len(data_files)

print("Number of items", N_items)

Number of items 1


In [19]:
sample_data = np.load(data_path+f'images_{data_files[0]}.npy')
sample_votes = np.load(data_path+f'votes_{data_files[0]}.npy')

In [20]:
sample_data.shape, sample_votes.shape

((46, 3, 400, 299), (46, 6))

In [21]:
# kl divergence loss
def kl_loss(y_true, y_pred):
    return np.sum(y_true * np.log(y_true / y_pred))

In [66]:
idx = 0
votes = sample_votes[idx]
votes = votes/np.sum(votes)

label_smoothing = 0.02
N_classes = len(votes)
votes = votes * (1 - label_smoothing) + label_smoothing / N_classes

pred = np.array([0,1,5,0,0,1])
pred = pred/np.sum(pred)
pred = pred * (1 - label_smoothing) + label_smoothing / N_classes

In [67]:
votes, pred

(array([0.00333333, 0.00333333, 0.98333333, 0.00333333, 0.00333333,
        0.00333333]),
 array([0.00333333, 0.14333333, 0.70333333, 0.00333333, 0.00333333,
        0.14333333]))

In [68]:
kl_loss(votes, np.array([1/6]*6)),kl_loss(votes, pred)

(1.68016942830601, 0.3044572683780671)

In [ ]:
# split data_files into training, validation, and test data (0.7, 0.2, 0.1)
np.random.seed(0)
np.random.shuffle(data_files)
train_files = data_files[:int(0.7*N_items)]
val_files = data_files[int(0.7*N_items):int(0.9*N_items)]
test_files = data_files[int(0.9*N_items):]


In [ ]:
train_data = CustomDataset(data_path, train_files)
train_loader = DataLoader(train_data, batch_size=1, shuffle=False, num_workers=num_workers)

valid_data = CustomDataset(data_path, val_files)
valid_loader = DataLoader(valid_data, batch_size=1, shuffle=False, num_workers=num_workers)

test_data = CustomDataset(data_path, test_files)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=num_workers)

In [ ]:
# specify the image classes
classes = [
        "seizure_vote",
        "lpd_vote",
        "gpd_vote",
        "lrda_vote",
        "grda_vote",
        "other_vote",
    ]
N_classes = len(classes)

In [ ]:
get_batch_transform = lambda x, y: (
    x[0, :],
    y[0, :],
)